In [1]:
import json
import util
import wowsims

This notebook uses wowsims to estimate the impact of each of the main Feral druid glyphs in BiS gear for phase two, three, and four.

In [2]:
# Load p2 as tbe base settings
p2_base = json.load(open('data/p2_feral.json'))

# Delete the database because our library bundles it
del p2_base['raid']['parties'][0]['players'][0]['database']

# Copy settings for p3/p4
p3_base = json.loads(json.dumps(p2_base))
p4_base = json.loads(json.dumps(p2_base))

# Replace the items with the p3 items
p3_base['raid']['parties'][0]['players'][0]['equipment']['items'] = json.load(open('data/p3_feral_items.json'))

# Same thing for p4 settings
p4_all = json.load(open('data/p4_feral.json'))
p4_base['raid']['parties'][0]['players'][0]['equipment']['items'] = p4_all['raid']['parties'][0]['players'][0]['equipment']['items']

Check and make sure that our baseline settings have the correct glyphs:

40902 -> Glyph of Rip

45604 -> Glyph of Savage Roar

40901 -> Glyph of Shred

In [3]:
# Equipped glyphs
[item for item in p2_base['raid']['parties'][0]['players'][0]['glyphs'].items() if item[0].startswith("major")]

major1,40902
major2,45604
major3,40901


In [4]:
# Sim p2, p3, and p4 with all Glyphs
p2 = wowsims.runSim(p2_base)
p3 = wowsims.runSim(p3_base)
p4 = wowsims.runSim(p4_base)
base_dps = [r['raidMetrics']['dps']['avg'] for r in [p2, p3, p4]]

In [5]:
# Helper function to run a sim without a glyph
def runWithoutGlyph(glyph):
    p2_settings = json.loads(json.dumps(p2_base))
    del p2_settings['raid']['parties'][0]['players'][0]['glyphs'][glyph]
    p3_settings = json.loads(json.dumps(p3_base))
    del p3_settings['raid']['parties'][0]['players'][0]['glyphs'][glyph]
    p4_settings = json.loads(json.dumps(p4_base))
    del p4_settings['raid']['parties'][0]['players'][0]['glyphs'][glyph]
    p2 = wowsims.runSim(p2_settings)
    p3 = wowsims.runSim(p3_settings)
    p4 = wowsims.runSim(p4_settings)
    return [r['raidMetrics']['dps']['avg'] for r in [p2, p3, p4]]

In [6]:
no_rip_dps = runWithoutGlyph('major1')
no_roar_dps = runWithoutGlyph('major2')
no_shred_dps = runWithoutGlyph('major3')

In [7]:
rip_loss = [base_dps[phase] - dps for (phase, dps) in enumerate(no_rip_dps)]
roar_loss = [base_dps[phase] - dps for (phase, dps) in enumerate(no_roar_dps)]
shred_loss = [base_dps[phase] - dps for (phase, dps) in enumerate(no_shred_dps)]
[('Glyph', 'P2', 'P3', 'P4'), ('Rip', rip_loss[0], rip_loss[1], rip_loss[2]), ('Roar', roar_loss[0], roar_loss[1], roar_loss[2]), ('Shred', shred_loss[0], shred_loss[1], shred_loss[2])]

Glyph,P2,P3,P4
Rip,128,132,137
Roar,237,272,375
Shred,212,232,252
